# Introduction

In [7]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# Load the Reuters dataset
(train_data, train_labels), (test_data, test_labels) = keras.datasets.reuters.load_data(num_words=10_000)

In [9]:
# Print dataset details
print(f"Training samples: {len(train_data)}, Test samples: {len(test_data)}")
print(f"Number of classes: {np.max(train_labels) + 1}") 

Training samples: 8982, Test samples: 2246
Number of classes: 46


In [10]:
# Decode a sample article
word_index = keras.datasets.reuters.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])

print("\nSample article (decoded):\n", decoded_review)

550378/550378 [==============================] - 1s 2us/step

Sample article (decoded):
 ? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3
